# GPT4o API로 2023 수능 국어 풀어보기

미완성인 관계로 구현한 부분까지만 제출하겠습니다!

1. 데이터 파악

2. 테스트 설계
- 1) 데이터를 어떻게 가공할 것인가?
    - 1-1. 지문 + 꼬리 문제 형태로 가공한다.
    - 1-2. 각 꼬리 문제의 5개 선지 시작 부분에 번호를 추가한다('[1]' + 기존 선지내용).

- 2) 프롬프트를 어떻게 넣을 것인가?
    - 2-1. Parsing을 위한 프롬프팅
        - 추론 과정 요구 프롬프트 + 정답 요구 프롬프트        
        - <<number>> 로 요구한다. (예를 들어 <<3>>)        
    
    - 2-2. 정답률을 높이기 위한 프롬프팅
        - 문제당 두 번의 문제 풀이를 독립적으로 진행한다.
        - 한 번은 zero-shot-CoT, 또 한 번은 plan-and-solve 를 적용하여 논리적 추론을 유도한다.
        - CoT 와 plan-and-solve에서 최종 도출한 정답이 동일할 경우 이를 제출하는 것으로 간주한다.
        - 정답이 다를 경우, 각 추론 기록을 참고하여 마지막 결정을 하는 추가 문제 풀이를 한다.

- 3) 정확도를 어떻게 측정할 것인가?
        - 문제 풀이마다 즉시 채점을 하고 정답일 시 total_score에 합산한다.
    

3. 구현
- 하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수를 `def prediction(problem)`이라는 signature로 만든다.
- `problem`은 json 형태의 문제.
- `def prediction` 함수를 모든 수능 국어 문제들에 대해서 돌린 후, 실제 정답과 비교하여 GPT-4의 점수를 계산하는 코드를 구현한다.

In [47]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


# [MY CODE] 데이터 파악
간단한 데이터로 진행하는 실습이므로, 별도의 파일에 준비한 json 데이터를 직접 참조하여 진행합니다.

[데이터 설명]
- id: 문제 지문의 id
- pragraph: 문제 지문
- type: 지문 타입 (화법과 작문, 문학, 비문학 등..)
- problems: 문제 정보들
    - question: 질문
    - choices: 보기(5개)
    - answer: 정답
    - score: 배점

In [48]:
import data

print(data.kice_data_2023[0],'\n')
print(data.kice_data_2023[0].keys(),'\n')
print(data.kice_data_2023[0]['problems'][0].keys())

{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통

# [MY CODE] 데이터 가공
1. 데이터를 어떻게 가공할 것인가?
    - 지문 + 꼬리 문제 형태로 가공한다.
        - 예를 들어 지문 하나에 꼬리 문제 2개라면, '지문 + 꼬리 문제1 + 보기 5개', '지문 + 꼬리 문제2 + 보기 5개'로 각각 나눈다.
    - choices의 각 선지 첫 부분에 번호를 추가한다('[1]' + 기존지문).

In [49]:
processed_data = []

# 데이터 가공
for entry in data.kice_data_2023:  # 각 지문(entry)을 순회
    for problem in entry["problems"]:  # 각 지문의 문제(problem)를 순회
        # 보기 가공
        choices = []
        for i, choice in enumerate(problem["choices"], start=1):
            choices.append(f"[{i}] {choice}")

        processed_entry = {
            "id": entry["id"],
            "type": entry["type"],
            "paragraph": entry["paragraph"],
            "question": problem["question"],
            "choices": choices,
            "answer": problem["answer"],
            "score": problem["score"],
        }

        # 결과 리스트에 추가
        processed_data.append(processed_entry)

# 출력
for key, value in processed_data[0].items():
    print(f"{key}: {value}")


id: 2023_11_KICE_1-3
type: 0
paragraph: 사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통한

# [MY CODE] 프롬프트 생성 함수
- 각 전략에 맞게 프롬프트를 생성하는 함수입니다.

1. <<숫자>> 로 요구한다. (예를 들어 <<3>>) 

2. 문제당 두 번의 문제 풀이를 독립적으로 진행한다.
    - 한 번은 zero-shot-CoT, 또 한 번은 plan-and-solve 를 적용하여 논리적 추론을 유도한다.

In [50]:
def create_prompt(paragraph, question, choices, strategy): # 문제, 지문, 보기, 전략
    
    # 수능 문제 + 지문 + 보기
    common_prompt = (
        f"Given the following passage:\n\n{paragraph}\n\n"
        f"Question: {question}\n\n"
        f"Choices:\n" + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
    )

    # 전략별 프롬프트
    if strategy == "cot":
        return (
            common_prompt
            + "\n\n추론 과정:\n"
            "Please provide a detailed, step-by-step reasoning process for your answer.\n\n"
            "정답 도출:\n"
            "Provide the final answer in the format <<number>>. (for example, <<1>> or <<2>>)"
        )
    elif strategy == "plan":
        return (
            common_prompt
            + "\n\nPlan:\n"
            "First, provide a short plan to approach this question.\n\n"
            "Solve:\n"
            "Based on the plan, Provide the final answer in the format <<number>>. (for example, <<1>> or <<2>>)"
        )


# [MY CODE] 정답 파싱함수
- 정답을 파싱하는 함수입니다.

In [51]:
# 정답 파싱
def extract_answer(response):
    import re
    if not response:
        return None
    match = re.search(r"<<(\d)>>", response)
    return int(match.group(1)) if match else None

# [MY CODE] OpenAI API 콜 함수

In [52]:
from openai import OpenAI

import openai
openai.api_key = '...'

client = OpenAI(api_key=openai.api_key)

print(client)

def call_openai_api(prompt, role):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": role},
                {"role": "user", "content": prompt},
            ],
            n=1,
            max_tokens=1024, # 출력 토큰 수는 1024로 제한
            temperature=0.2,
        )

        # 메시지 content만 추출
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API call failed: {e}")
        return None

# [MY CODE] 문제풀이 함수 (prediction 함수)

1. 한 문제당 두 번의 풀이 + (optional) 추가 풀이를 진행합니다.
    - 한 번은 zero-shot-CoT, 또 한 번은 plan-and-solve 를 적용하여 논리적 추론을 유도한다.
    - CoT 와 plan-and-solve에서 최종 도출한 정답이 동일할 경우 이를 최종 정답으로 간주한다.
    - 각 정답이 다를 경우, 추론 기록을 참고하여 마지막 결정을 하는 추가 문제 풀이를 한다.

2. 채점 방식
    - 문제 풀이마다 즉시 채점을 한다. 오답일 경우에 감점(minus) 변수에 가점한다. 마지막에 100 - minus 으로 최종 점수를 도출한다.
    - 복기를 위해, 오답일 경우 wrong_solves 리스트에 question, solve (모델 답변), answer (실제 정답), score를 저장한다.

In [53]:
# GPT-4o 예측 함수
def prediction(problem):
    """
    문제를 JSON 형식으로 받아 GPT-4o를 통해 정답을 예측하는 함수.
    문제 풀이는 Zero-shot-CoT 및 Plan-and-solve 방식을 사용.
    """
    question = problem["question"]
    choices = problem["choices"]
    answer = problem["answer"]  # 실제 정답 (채점에 사용)

    # 프롬프트 - 문제 지문
    paragraph = problem.get("paragraph", "No paragraph provided.")

    # 프롬프트 - role 정의
    assistant_role = "You are a highly experienced educator specializing in Korean language and literature, with extensive knowledge of the Korean College Scholastic Ability Test (CSAT). Your expertise includes analyzing and solving high school-level Korean questions in areas such as literature, nonfiction, and writing."
    
    # 프롬프트 - zero-shot-cot 전략
    user_prompt_cot = create_prompt(paragraph, question, choices, "cot")

    # 프롬프트 - plan-and-solve 전략
    user_prompt_plan = create_prompt(paragraph, question, choices, "plan")

    # OpenAI API 요청
    cot_result = call_openai_api(user_prompt_cot, assistant_role)
    plan_result = call_openai_api(user_prompt_plan, assistant_role)

    # 정답만 추출
    cot_answer = extract_answer(cot_result)
    plan_answer = extract_answer(plan_result)

    print('\n\n', question, '\n [cot_result]:', cot_result, '\n [cot_answer]:', cot_answer)
    print('\n\n', question, '\n [plan_result]:', plan_result, '\n [plan_answer]:', plan_answer)

    # 결과 처리
    if cot_answer == plan_answer:
        final_answer = cot_answer
        print(question, 'final_answer(만장일치):', final_answer)
    else:
        # 추가 풀이 요청 (모델 간 도출한 답 불일치 시)
        additional_prompt = (
            f"Previous answers were not unanimous.\n\n"
            f"Passage: {paragraph}\n\n"
            f"Question: {question}\n\n"
            f"Choices:\n"
            + "\n".join([f"[{i+1}] {choice}" for i, choice in enumerate(choices)])
            + "\n\n"
            "Earlier reasoning:\n"
            f"Chain of Thought reasoning:\n{cot_result}\n\n"
            f"Plan-and-Solve reasoning:\n{plan_result}\n\n"
            "Updated reasoning:\n"
            "Please reconsider the previous reasoning and provide a detailed explanation for your final answer.\n\n"
            "Final answer:\n"
            "Provide the final answer in the format <<number>> (for example, <<1>> or <<2>> or <<3>> ...)"
        )

        additional_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": assistant_role},
                {"role": "user", "content": additional_prompt},
            ],
            n=1,
            max_tokens=300,
            temperature=0.1,
        )
        final_answer = extract_answer(additional_response.choices[0].message.content)
        print(question, '\n\nfinal_ansewr(다시 풀기):', final_answer)

    # 채점
    correct = final_answer == answer
    print('\n\n정답 여부:', correct)
    return {
        "question": question,
        "choices": choices,
        "model_answer": final_answer,
        "correct_answer": answer,
        "score": problem["score"],
        "correct": correct,
    }



# [MY CODE] 문제풀이 실행 및 평가 함수 (evaluate)

In [54]:
# 전체 데이터 처리 및 점수 계산
def evaluate_model(problems):
    results = []
    total_score = 0
    minus_score = 0

    for problem in problems:
        result = prediction(problem)
        results.append(result)
        if result["correct"]:
            total_score += result["score"]
        else:
            minus_score += result["score"]

    # 최종 점수
    return {"results": results, "total_score": total_score}


# 평가 실행
problems = processed_data
evaluation = evaluate_model(problems)
print(evaluation)



 윗글의 내용과 일치하지 않는 것은? 
 [cot_result]: To determine which statement does not align with the passage, let's analyze each option against the content of the passage:

1. **Option [1]:** "같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다."
   - The passage states that even if readers read the same book, they can construct different meanings due to various factors such as background knowledge and perspectives. This aligns with the passage, particularly the part that mentions 독자 요인 (reader factors) and 상황 요인 (situational factors) leading to different interpretations.

2. **Option [2]:** "다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다."
   - The passage discusses how sharing interpretations with other readers, either through agreement or recognizing differences, can deepen and expand one's understanding. This is consistent with the passage.

3. **Option [3]:** "독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다."
   - The passage explicitly mentions that readers can encounter and understand various lives they haven't directly exp

In [60]:
# evaluation 결과를 순회하면서 출력하고, 총점을 계산
total_score = 0
num = 1
for result in evaluation["results"]:
    print(f"Q{num}. {result['question']} ({result['score']}점) \n-  Model Answer: {result['model_answer']} | Correct: {result['correct']}")
    
    if result["correct"]:
        total_score += result["score"]
    
    num += 1
    

print(f"Total Score: {total_score}")

Q1. 윗글의 내용과 일치하지 않는 것은? (2점) 
-  Model Answer: 4 | Correct: True
Q2. 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은? (3점) 
-  Model Answer: 4 | Correct: False
Q3. 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은? (2점) 
-  Model Answer: 1 | Correct: True
Q4. (가)와 (나)에 대한 설명으로 가장 적절한 것은? (2점) 
-  Model Answer: 4 | Correct: True
Q5. [A]에 대한 이해로 적절하지 않은 것은? (2점) 
-  Model Answer: 5 | Correct: True
Q6. ㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은? (2점) 
-  Model Answer: 3 | Correct: True
Q7. ㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은? (2점) 
-  Model Answer: 2 | Correct: True
Q8. (가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 않은 것은? (3점) 
-  Model Answer: 3 | Correct: False
Q9. 문맥상 ⓐ～ⓔ와 바꾸어 쓰기에 적절하지 않은 것은? (2점) 
-  Model Answer: 2 | Correct: True
Q10. 윗글의 내용과 일치하지 않는 것은? (2점) 
-  Model Answer: 4 | Correct: True
Q11. ㉠에 대한 이해로 가장 적절한 것은? (2점) 
-  Model Answer: 5 | Correct: True
Q12. 윗글을 바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은? [3점] (3점) 
-  Model Answer: 3 | Correct: False
Q13. 문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은? (2점) 
-  Model Ans